In [40]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    Activation,
    Input,
    BatchNormalization,
    Convolution2D,
    Conv1D,
    Flatten,
    Embedding,
    MaxPooling1D,
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping # early stopping
from tensorflow.keras.regularizers import l1
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [91]:
print(tf.VERSION)
print(keras.__version__)

1.12.0
2.1.6-tf


In [93]:
max_words = 1000
batch_size = 32
epochs = 5
kernel_size = 3
conv_depth = 32
l1_lambda = 0.0001

In [75]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [76]:
print(newsgroups_train["data"][1])

From: guykuo@carson.u.washington.edu (Guy Kuo)
Subject: SI Clock Poll - Final Call
Summary: Final call for SI clock reports
Keywords: SI,acceleration,clock,upgrade
Article-I.D.: shelley.1qvfo9INNc3s
Organization: University of Washington
Lines: 11
NNTP-Posting-Host: carson.u.washington.edu

A fair number of brave souls who upgraded their SI clock oscillator have
shared their experiences for this poll. Please send a brief message detailing
your experiences with the procedure. Top speed attained, CPU rated speed,
add on cards and adapters, heat sinks, hour of usage per day, floppy disk
functionality with 800 and 1.4 m floppies are especially requested.

I will be summarizing in the next two days, so please add to the network
knowledge base if you have done the clock upgrade and haven't answered this
poll. Thanks.

Guy Kuo <guykuo@u.washington.edu>



In [77]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(newsgroups_train["data"])

In [78]:
x_train = tokenizer.texts_to_matrix(newsgroups_train["data"], mode='binary')
x_test = tokenizer.texts_to_matrix(newsgroups_test["data"], mode='binary')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (11314, 1000)
x_test shape: (7532, 1000)


In [79]:
x_train

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 1., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [80]:
num_classes = np.max(newsgroups_train["target"]) + 1
print(num_classes, 'classes')

20 classes


In [81]:
print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = keras.utils.to_categorical(newsgroups_train["target"], num_classes)
y_test = keras.utils.to_categorical(newsgroups_test["target"], num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (11314, 20)
y_test shape: (7532, 20)


In [82]:
y_train[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)

In [127]:
model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(max_words,)))
model.add(Dense(512))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization(axis=1))
model.add(Dense(num_classes))
model.add(BatchNormalization(axis=1))
model.add(Activation('softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

In [128]:
# Old
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5)],

)

Train on 10182 samples, validate on 1132 samples
Epoch 1/5
10182/10182 [==============================] - 12s 1ms/step - loss: 1.8884 - acc: 0.4841 - val_loss: 1.2369 - val_acc: 0.7588
Epoch 2/5
10182/10182 [==============================] - 7s 707us/step - loss: 1.1968 - acc: 0.7074 - val_loss: 0.8702 - val_acc: 0.7686
Epoch 3/5
10182/10182 [==============================] - 7s 712us/step - loss: 0.9869 - acc: 0.7585 - val_loss: 0.7935 - val_acc: 0.7836
Epoch 4/5
10182/10182 [==============================] - 8s 780us/step - loss: 0.8535 - acc: 0.7893 - val_loss: 0.7517 - val_acc: 0.7898
Epoch 5/5
10182/10182 [==============================] - 7s 676us/step - loss: 0.7638 - acc: 0.8107 - val_loss: 0.7318 - val_acc: 0.7880


In [129]:
# Old
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7532/7532 [==============================] - 1s 106us/step


Test score: 1.1633422013235675
Test accuracy: 0.674057355284121


In [130]:
model = Sequential()
model.add(BatchNormalization(axis=1, input_shape=(max_words,)))
model.add(Dense(512, activation='sigmoid', kernel_initializer='glorot_uniform'))
model.add(BatchNormalization(axis=1))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.add(BatchNormalization(axis=1))
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

In [131]:
# New
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5)],

)

Train on 10182 samples, validate on 1132 samples
Epoch 1/5
10182/10182 [==============================] - 9s 925us/step - loss: 8.3116 - acc: 0.1510 - val_loss: 5.7455 - val_acc: 0.3277
Epoch 2/5
10182/10182 [==============================] - 6s 634us/step - loss: 6.9506 - acc: 0.3409 - val_loss: 7.0177 - val_acc: 0.3587
Epoch 3/5
10182/10182 [==============================] - 6s 638us/step - loss: 5.2851 - acc: 0.3503 - val_loss: 5.1439 - val_acc: 0.4099
Epoch 4/5
10182/10182 [==============================] - 7s 722us/step - loss: 5.0000 - acc: 0.3741 - val_loss: 4.6036 - val_acc: 0.4293
Epoch 5/5
10182/10182 [==============================] - 7s 652us/step - loss: 4.8144 - acc: 0.3632 - val_loss: 5.6799 - val_acc: 0.3118


In [132]:
# New
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7532/7532 [==============================] - 1s 104us/step


Test score: 6.089772275579273
Test accuracy: 0.28120021242697824


In [34]:
batch_size = 100
max_words = 1000
epochs = 10
vocab_size = 1000

In [50]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups()

encoded_newsgroups = [one_hot(d, vocab_size) for d in newsgroups.data]
encoded_newsgroups = pad_sequences(encoded_newsgroups, maxlen=max_length, padding='post')

# encoded_target = keras.utils.to_categorical(newsgroups.target)
encoded_target = newsgroups.target

num_classes = np.max(encoded_target) + 1
max_length = max(map(len, encoded_newsgroups))


In [49]:
newsgroups.target

array([7, 4, 4, ..., 3, 1, 8])

In [51]:
newsgroups_train, newsgroups_test, y_train, y_test = train_test_split(
    encoded_newsgroups, encoded_target, test_size=0.33, random_state=42
)

In [61]:
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Conv1D(40, 3, activation='relu',))
model.add(MaxPooling1D())
model.add(Conv1D(20, 3, activation='relu',))
model.add(MaxPooling1D())
model.add(Flatten())
# model.add(Dense(
#     512,
#     activation='sigmoid',
#     kernel_initializer='glorot_uniform',
#     input_shape=(max_words,)
# ))
# model.add(Dropout(0.1))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
)

In [62]:
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 16333, 8)          8000      
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 16331, 40)         1000      
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 8165, 40)          0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 8163, 20)          2420      
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 4081, 20)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 81620)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 20)                1632420   
Total para

In [63]:
from tensorflow.keras. import model_to_dot
from IPython.display import Image

Image(model_to_dot(model, show_shapes=True).create(prog='dot', format='png'))

ModuleNotFoundError: No module named 'tensorflow.keras.utils.visualize_util'

In [57]:
# X_train = np.expand_dims(x_train, axis=2)
# X_test = np.expand_dims(x_test, axis=2)

# X_train = np.reshape(x_train, (1, x_train.shape[0], x_train.shape[1]))
# X_test = np.reshape(x_test, (1, x_test.shape[0], x_test.shape[1]))

# X_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
# X_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

history = model.fit(
    newsgroups_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_split=0.1,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5)],

)

ValueError: Error when checking target: expected dense_5 to have shape (20,) but got array with shape (1,)

In [ ]:
score = model.evaluate(X_test, y_test,
                       batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [199]:
max_words

1000

In [185]:
X_test.shape

(7532, 1000, 1)